<table align="left"><td><a target="_blank" href="https://colab.research.google.com/github/superannotateai/model-deployment-tutorials/blob/main/OAK/SuperAnnotate_OAK_YOLOv4_tiny_Deployment.ipynb"><img src="https://user-images.githubusercontent.com/25985824/104791629-6e618700-5769-11eb-857f-d176b37d2496.png" height="32" width="32"> Try in Google Colab</a></td></table>

Source: https://github.com/superannotateai/model-deployment-tutorials/blob/main/OAK/SuperAnnotate_OAK_YOLOv4_tiny_Deployment.ipynb

# 1. Install Prerequisites 
(Please click on **RESTART RUNTIME** button when it appears in the output of this code block)



In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git
!git clone https://github.com/GotG/yolotinyv3_medmask_demo
!git clone https://github.com/TNTWEN/OpenVINO-YOLOV4.git

%cd darknet
import re
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!make
!chmod +x ./darknet
%cd ../

!sudo apt-get install -y pciutils cpio
!sudo apt autoremove
!wget http://registrationcenter-download.intel.com/akdlm/irc_nas/16803/l_openvino_toolkit_p_2020.4.287.tgz
path = "l_openvino_toolkit_p_2020.4.287.tgz"
!tar xf "{path}"

%cd l_openvino_toolkit_p_2020.4.287/
!./install_openvino_dependencies.sh && \
    sed -i 's/decline/accept/g' silent.cfg && \
    ./install.sh --silent silent.cfg

!bash /content/l_openvino_toolkit_p_2020.4.287/install_openvino_dependencies.sh
!bash /opt/intel/openvino/deployment_tools/model_optimizer/install_prerequisites/install_prerequisites.sh

!pip install superannotate
!pip install google-resumable-media==0.5.0

Cloning into 'darknet'...
remote: Enumerating objects: 15097, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 15097 (delta 8), reused 9 (delta 2), pack-reused 15072
Receiving objects: 100% (15097/15097), 13.48 MiB | 24.61 MiB/s, done.
Resolving deltas: 100% (10253/10253), done.
Cloning into 'yolotinyv3_medmask_demo'...
remote: Enumerating objects: 1733, done.
remote: Total 1733 (delta 0), reused 0 (delta 0), pack-reused 1733
Receiving objects: 100% (1733/1733), 208.88 MiB | 38.87 MiB/s, done.
Resolving deltas: 100% (120/120), done.
Checking out files: 100% (1537/1537), done.
Cloning into 'OpenVINO-YOLOV4'...
remote: Enumerating objects: 667, done.
remote: Counting objects: 100% (462/462), done.
remote: Compressing objects: 100% (419/419), done.
remote: Total 667 (delta 108), reused 337 (delta 29), pack-reused 205
Receiving objects: 100% (667/667), 48.14 MiB | 40.34 MiB/s, done.
Resolving deltas: 100% (219/219), done.
/c

ERROR: google-cloud-storage 1.38.0 has requirement google-resumable-media<2.0dev,>=1.2.0, but you'll have google-resumable-media 0.5.0 which is incompatible.
ERROR: google-cloud-bigquery 1.21.0 has requirement google-resumable-media!=0.4.0,<0.5.0dev,>=0.3.1, but you'll have google-resumable-media 0.5.0 which is incompatible.
  Found existing installation: google-resumable-media 1.3.0
    Uninstalling google-resumable-media-1.3.0:
      Successfully uninstalled google-resumable-media-1.3.0


# 2. Setup your SuperAnnotate token, project names of trianing images and desired output model name

In [ ]:
TOKEN = "token"
PROJECT_NAMES = ["Food Quality Project"]
OUTPUT_MODEL = "food-quality-yolov4-v1"

# 3. Download and preprocess the data for training

In [ ]:
import superannotate as sa
import json
import os 
from shutil import copy, make_archive
import re

proj_folder = dict(zip(PROJECT_NAMES, ["proj_" + str(i) for i in range(len(PROJECT_NAMES))]))
token_json = {"token": TOKEN}
with open('sa_config.json', 'w') as f:
  json.dump(token_json, f)

sa.init('sa_config.json')

for project_name, folder_name in proj_folder.items():
  if not os.path.exists('./training_data/' + folder_name):
    os.makedirs('training_data/' + folder_name)
  completed_images = sa.search_images(project_name, annotation_status="Completed")
  for completed_image in completed_images:
    sa.download_image(project_name, completed_image, local_dir_path='./training_data/' + folder_name)
  export = sa.prepare_export(project_name, annotation_statuses=["Completed"])
  sa.download_export(project_name, export, './training_data/' + folder_name)

SA-PYTHON-SDK - INFO - Downloaded image __opt__aboutcom__coeus__resources__content_migration__simply_recipes__uploads__2006__08__HT-Cut-A-Mango-LEAD-2-924f2172b to training_data/proj_0/__opt__aboutcom__coeus__resources__content_migration__simply_recipes__uploads__2006__08__HT-Cut-A-Mango-LEAD-2-924f2172b.
SA-PYTHON-SDK - INFO - Downloaded image 000001.jpg to training_data/proj_0/000001.jpg.
SA-PYTHON-SDK - INFO - Downloaded image 000002.jpg to training_data/proj_0/000002.jpg.
SA-PYTHON-SDK - INFO - Downloaded image 000003.jpg to training_data/proj_0/000003.jpg.
SA-PYTHON-SDK - INFO - Downloaded image 000004.jpg to training_data/proj_0/000004.jpg.
SA-PYTHON-SDK - INFO - Downloaded image 000005.jpg to training_data/proj_0/000005.jpg.
SA-PYTHON-SDK - INFO - Downloaded image 000006.jpg to training_data/proj_0/000006.jpg.
SA-PYTHON-SDK - INFO - Downloaded image 000007.jpg to training_data/proj_0/000007.jpg.
SA-PYTHON-SDK - INFO - Downloaded image 000008.jpg to training_data/proj_0/000008.jp

FileNotFoundError: ignored

In [ ]:
class_names = set()
for folder_name in proj_folder.values():
  with open('/content/training_data/' + folder_name +'/classes/classes.json', 'r') as f:
    proj_class_data = json.load(f)
  for class_entry in proj_class_data:
    class_names.add(class_entry["name"])

class_names = list(class_names)
class_names_yolo_id = dict(zip(class_names, range(len(class_names))))

if not os.path.exists('/content/training_data/yolo_train'):
  os.mkdir('/content/training_data/yolo_train')

for folder_name in proj_folder.values():
  folder_base = '/content/training_data/' + folder_name
  folder_content = os.listdir(folder_base)
  for folder_item_path in folder_content:
    if folder_item_path.endswith("classes"):
      continue
    elif folder_item_path.endswith("json"):
      yolo_annot_file_path = '/content/training_data/yolo_train/' + folder_name + '_' + os.path.splitext(folder_item_path.split("___")[0])[0] + '.txt'
      with open(folder_base + '/' + folder_item_path, 'r') as f:
        annot_data = json.load(f)
      img_h, img_w = annot_data["metadata"]["height"], annot_data["metadata"]["width"]
      box_data = []
      for annot_inst in annot_data["instances"]:
        if annot_inst["type"] != "bbox":
          continue
        points = annot_inst["points"]
        x1, x2, y1, y2 = points["x1"], points["x2"], points["y1"], points["y2"]
        center_x = str(((x1 + x2) / 2) / img_w)
        center_y = str(((y1 + y2) / 2) / img_h)
        width = str((max(x1, x2) - min(x1, x2)) / img_w)
        height = str((max(y1, y2) - min(y1, y2)) / img_h)
        inst_class_id = str(class_names_yolo_id[annot_inst["className"]])
        box_data.append(inst_class_id + ' ' + center_x + ' ' + center_y + ' ' + width + ' ' + height + '\n')
      with open(yolo_annot_file_path, 'w') as f:
        for box_inst in box_data:
          f.write(box_inst)
    else:
      copy(folder_base + '/' + folder_item_path, '/content/training_data/yolo_train/' + folder_name + '_' + folder_item_path)

labels_path = '/content/yolotinyv3_medmask_demo/obj.names'

with open(labels_path, 'w') as f:
  for class_name, yolo_id in class_names_yolo_id.items():
    f.write(class_name + '\n')

objdata = '/content/yolotinyv3_medmask_demo/obj.data'
with open(objdata) as f:
    s = f.read()

num_classes = len(class_names_yolo_id.keys())
s = re.sub('classes= \d*','classes = ' + str(num_classes),s)

with open(objdata, 'w') as f:
  f.write(s)

In [ ]:
%cd /content/yolotinyv3_medmask_demo/
!python3 folder2textYolo.py 90 5 /content/training_data/yolo_train

/content/yolotinyv3_medmask_demo
Your image file extension is: .png
Number of images: 1127
Number of images used for training 1015
Number of images used for validation 56
Number of images used for testing 56


# 4. Train the model

In [ ]:
max_batch= num_classes * 3000 #recommended 2000
step1 = 0.8 * max_batch
step2 = 0.9 * max_batch
subdivisions = 4
num_filters = (num_classes + 5) * 3

cfg_file = '/content/yolotinyv3_medmask_demo/yolov4-tiny.cfg'

with open(cfg_file) as f:
    s = f.read()

s = re.sub('subdivisions=\d*','subdivisions='+str(subdivisions),s)
s = re.sub('max_batches = \d*','max_batches = '+str(max_batch),s)
s = re.sub('steps=\d*,\d*','steps='+"{:.0f}".format(step1)+','+"{:.0f}".format(step2),s)
s = re.sub('classes=\d*','classes='+str(num_classes),s)
s = re.sub('pad=1\nfilters=\d*','pad=1\nfilters='+"{:.0f}".format(num_filters),s)

with open(cfg_file, 'w') as f:
  f.write(s)

%cd /content/darknet/

!./darknet detector train /content/yolotinyv3_medmask_demo/obj.data /content/yolotinyv3_medmask_demo/yolov4-tiny.cfg /content/yolotinyv3_medmask_demo/yolov4-tiny.conv.29 -dont_show -ext_output -map

Streaming output truncated to the last 5000 lines.
 total_bbox = 461624, rewritten_bbox = 0.118928 % 

 (next mAP calculation at 5800 iterations) 

 Tensor Cores are used.
 Last accuracy mAP@0.5 = 85.69 %, best = 90.99 % 
 5741: 0.052856, 0.083774 avg loss, 0.000026 rate, 0.673256 seconds, 367424 images, 0.068041 hours left
Loaded: 0.000058 seconds
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.916996), count: 16, class_loss = 0.032144, iou_loss = 0.059553, total_loss = 0.091698 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 461640, rewritten_bbox = 0.118924 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.892200), count: 19, class_loss = 0.129532, iou_loss = 0.116921, total_loss = 0.246453 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.786392), c

# 5. Generate and download the files needed for the OAK-D device

In [ ]:
%tensorflow_version 1.x
from google.colab import files
import requests

files.download('/content/darknet/backup/yolov4-tiny_best.weights')

%cd /content/OpenVINO-YOLOV4/
!python3 convert_weights_pb.py --class_names /content/yolotinyv3_medmask_demo/obj.names --data_format NHWC --weights_file /content/darknet/backup/yolov4-tiny_best.weights --tiny

tiny_yolo_json = '/content/OpenVINO-YOLOV4/yolo_v4_tiny.json'
with open(tiny_yolo_json) as f:
    s = f.read()
s = re.sub('"classes": \d*','"classes": ' + str(num_classes),s)
with open(tiny_yolo_json, 'w') as f:
  f.write(s)

!source /opt/intel/openvino/bin/setupvars.sh && \
    python /opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model /content/OpenVINO-YOLOV4/frozen_darknet_yolov4_model.pb \
--transformations_config /content/OpenVINO-YOLOV4/yolo_v4_tiny.json  \
--batch 1 \
--data_type FP16 \
--reverse_input_channel \
--output_dir /content/yolotinyv3_medmask_demo/TinyIR

os.mkdir("/content/" + OUTPUT_MODEL)
blob_dir = "/content/" + OUTPUT_MODEL

binfile = "/content/yolotinyv3_medmask_demo/TinyIR/frozen_darknet_yolov4_model.bin"
xmlfile = "/content/yolotinyv3_medmask_demo/TinyIR/frozen_darknet_yolov4_model.xml"
"""
url = "http://69.164.214.171:8083/compile"
params = {
    'version': '2020.1'
}
payload = {
    'compile_type': 'myriad', 
    'compiler_params': '-ip U8 -VPU_MYRIAD_PLATFORM VPU_MYRIAD_2480 -VPU_NUMBER_OF_SHAVES 6 -VPU_NUMBER_OF_CMX_SLICES 6'
}

model_files = {
    'definition': open(xmlfile, 'rb'),
    'weights': open(binfile, 'rb')
}
response = requests.post(url, data=payload, files=model_files, params=params)
blobname = OUTPUT_MODEL + '.blob.sh14cmx14NCE1'
with open(blob_dir + '/' + blobname, 'wb') as f:
  f.write(response.content)
"""

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/content/OpenVINO-YOLOV4
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0615 14:42:46.146373 140242952374144 module_wrapper.py:139] From convert_weights_pb.py:38: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


W0615 14:42:46.148550 140242952374144 module_wrapper.py:139] From convert_weights_pb.py:40: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

Instructions for updating:
Please use `layer.__call__` method instead.
W0615 14:42:46.153270 140242952374144 deprecation.py:323] From /tensorflow-1.15.2/python3.7/tensorflow_core/contrib/layers/python/layers/layers.py:1057: Layer.apply (from tensor

'\nurl = "http://69.164.214.171:8083/compile"\nparams = {\n    \'version\': \'2020.1\'\n}\npayload = {\n    \'compile_type\': \'myriad\', \n    \'compiler_params\': \'-ip U8 -VPU_MYRIAD_PLATFORM VPU_MYRIAD_2480 -VPU_NUMBER_OF_SHAVES 6 -VPU_NUMBER_OF_CMX_SLICES 6\'\n}\n\nmodel_files = {\n    \'definition\': open(xmlfile, \'rb\'),\n    \'weights\': open(binfile, \'rb\')\n}\nresponse = requests.post(url, data=payload, files=model_files, params=params)\nblobname = OUTPUT_MODEL + \'.blob.sh14cmx14NCE1\'\nwith open(blob_dir + \'/\' + blobname, \'wb\') as f:\n  f.write(response.content)\n'

In [ ]:
%cd /content/
!wget https://raw.githubusercontent.com/luxonis/depthai/main/resources/nn/tiny-yolo-v3/model.yml
!wget https://raw.githubusercontent.com/luxonis/depthai/main/resources/nn/tiny-yolo-v3/tiny-yolo-v3.json

try:
  with open('/content/model.yml', 'r') as f:
      s = f.read()
  s = re.sub('v3','v4',s)
  with open('/content/' + OUTPUT_MODEL + '/model.yml', 'w') as f:
      f.write(s)
  with open('tiny-yolo-v3.json', 'r') as f:
    settings_json = json.load(f)
  settings_json["nn_config"]["NN_specific_metadata"]["classes"] = num_classes
  settings_json["mappings"]["labels"] = list(class_names_yolo_id.keys())
  with open('/content/' + OUTPUT_MODEL + '/' + OUTPUT_MODEL + '.json', 'w') as f:
    json.dump(settings_json, f)
  make_archive(OUTPUT_MODEL, 'zip', OUTPUT_MODEL)
  files.download(OUTPUT_MODEL + '.zip')
except:
  print("ERROR")

/content
--2021-06-15 15:00:35--  https://raw.githubusercontent.com/luxonis/depthai/main/resources/nn/tiny-yolo-v3/model.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 592 [text/plain]
Saving to: ‘model.yml.1’

model.yml.1         100%[===================>]     592  --.-KB/s    in 0s      

2021-06-15 15:00:36 (34.3 MB/s) - ‘model.yml.1’ saved [592/592]

--2021-06-15 15:00:36--  https://raw.githubusercontent.com/luxonis/depthai/main/resources/nn/tiny-yolo-v3/tiny-yolo-v3.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>